In [1]:
pip install pandas plotly nbformat

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
from pathlib import Path

In [3]:
actuals_path   = "C:\\Users\\airjo\\Desktop\\Coding\\Time-Series Forecasting for Supply Chain Logistics\\data\\processed\\merged_data.csv"
forecast_path  = "C:\\Users\\airjo\\Desktop\\Coding\\Time-Series Forecasting for Supply Chain Logistics\\data\\processed\\forecast_results.csv"

In [4]:
actuals = pd.read_csv(actuals_path, parse_dates=["Date"])
forecast = pd.read_csv(forecast_path, parse_dates=["ds"])

In [5]:
actuals = actuals.rename(columns={"Date": "ds", "Weekly_Sales": "actual_sales"})
forecast = forecast.rename(columns={"yhat": "forecast_sales"})

In [6]:
df = pd.merge(
    actuals[["Store", "ds", "actual_sales"]],
    forecast[["Store", "ds", "forecast_sales"]],
    on=["Store", "ds"],
    how="inner",
)

In [7]:
df["abs_error"] = (df["actual_sales"] - df["forecast_sales"]).abs()
df["pct_error"] = df["abs_error"] / df["actual_sales"] * 100

In [8]:
df.head()
df.Store.unique()     # list of store IDs
df.ds.min(), df.ds.max()

(Timestamp('2010-02-05 00:00:00'), Timestamp('2012-10-26 00:00:00'))

In [9]:
import plotly.express as px

In [10]:
df_melted = df.melt(
    id_vars=["Store", "ds"], 
    value_vars=["actual_sales", "forecast_sales"],
    var_name="kind", 
    value_name="sales",
)

In [ ]:
import plotly.express as px

# 1. Create a “master” DataFrame with a “kind” column for stacking actual vs forecast
df_melted = df.melt(
    id_vars=["Store", "ds"], 
    value_vars=["actual_sales", "forecast_sales"],
    var_name="kind", 
    value_name="sales",
)
# Now df_melted has rows like:
# | Store | ds         | kind            | sales |
# | 1     | 2010-02-05 | actual_sales    | 1500000 |
# | 1     | 2010-02-05 | forecast_sales  | 1523000 |
# | …     | …          | …               | …       |

# 2. Create a line chart with Plotly Express
fig1 = px.line(
    df_melted,
    x="ds",
    y="sales",
    color="kind",
    facet_col=None,
    facet_row=None,
    line_dash="kind",   # optional: differentiate lines by dash style
    labels={
        "ds": "Week (Date)",
        "sales": "Sales ($)",
        "kind": "Series"
    },
    title="Actual vs. Forecast: Weekly Sales for All Stores",
    hover_data={"Store": True, "kind": True},
)

# 3. Add a dropdown so we can filter by Store
fig1.update_layout(
    updatemenus=[{
        "buttons": [
            {
                "method": "update",
                "label": f"Store {s}",
                "args": [
                    {"visible": df_melted["Store"] == s},
                    {"title": f"Actual vs. Forecast Sales (Store {s})"}
                ],
            }
            for s in sorted(df_melted["Store"].unique())
        ],
        "direction": "down",
        "pad": {"r": 10, "t": 10},
        "showactive": True,
        "x": 0.1,
        "xanchor": "left",
        "y": 1.15,
        "yanchor": "top",
    }]
)

# 4. Style: put actual_sales as a solid line, forecast_sales as dashed line
fig1.update_traces(
    selector=dict(name="actual_sales"),
    line=dict(color="royalblue", width=3, dash="solid")
)
fig1.update_traces(
    selector=dict(name="forecast_sales"),
    line=dict(color="darkorange", width=3, dash="dash")
)

In [ ]:
fig1.show()

In [ ]:
# 1. Compute MAE per store
mae_by_store = (
    df.groupby("Store")["abs_error"]
      .mean()
      .reset_index()
      .rename(columns={"abs_error": "MAE"})
)

# 2. Sort stores descending by MAE
mae_by_store = mae_by_store.sort_values("MAE", ascending=False)

# 3. Bar chart with Plotly Express
fig2 = px.bar(
    mae_by_store,
    x="Store",
    y="MAE",
    color="MAE", 
    color_continuous_scale="Reds",
    labels={"MAE": "Mean Absolute Error", "Store": "Store ID"},
    title="Average Absolute Forecast Error (MAE) by Store",
)

# 4. Add hover text showing error in dollars and percentage
# We'll merge a pct_error_by_store for extra info
pct_err = (
    df.groupby("Store")["pct_error"]
      .mean()
      .reset_index()
      .rename(columns={"pct_error": "AvgPctError"})
)
mae_by_store = mae_by_store.merge(pct_err, on="Store")

fig2 = px.bar(
    mae_by_store,
    x="Store",
    y="MAE",
    color="MAE",
    color_continuous_scale="Reds",
    labels={"MAE": "MAE ($)", "Store": "Store ID"},
    title="Average Absolute Forecast Error (MAE) by Store",
    hover_data={
        "MAE": ":,.0f",          # show MAE with comma separators
        "AvgPctError": ":.2f"    # show AvgPctError as percentage with 2 decimals
    },
)
fig2.update_layout(coloraxis_showscale=False)
fig2.show()


In [ ]:
# 1. Add a week number column
df["week_of_year"] = df["ds"].dt.isocalendar().week.astype(int)  # ISO week (1–52)

# 2. Pivot to create a matrix (Store × week) of abs_error
heatmap_df = df.pivot_table(
    index="Store", 
    columns="week_of_year", 
    values="abs_error", 
    aggfunc="mean",   # each store-week should already be unique; mean is just a formality
    fill_value=0
)

# 3. Convert the pivot to “long” form for Plotly
heatmap_long = heatmap_df.reset_index().melt(
    id_vars="Store", 
    var_name="Week", 
    value_name="AbsError"
)

# 4. Plotly Express heatmap
fig3 = px.density_heatmap(
    heatmap_long,
    x="Week",
    y="Store",
    z="AbsError",
    color_continuous_scale="OrRd",
    labels={"AbsError": "Absolute Error ($)", "Store": "Store ID"},
    title="Weekly Absolute Forecast Error by Store (Heatmap)",
)

# 5. Style axes
fig3.update_xaxes(dtick=4)  # show tick every 4 weeks
fig3.update_yaxes(autorange="reversed")  # show store 1 at top (optional)

fig3.show()
